# Week 1 - Interpreting Image Classifiers

Welcome to the week 1 project of the Interpreting Machine Learning Models course! We are excited to help you unravel the mysteries behind machine learning algorithms.

## Introduction - Week 1 Challenge

It's 2050 and a mysterious virus has caused the global cat population to become hilariously clumsy and forgetful. People can often be seen watching in amusement as their feline companions stumble into walls, accidentally headbutt their own tails, and knock over everything in their way!

<center><img src='https://media.tenor.com/FJgMcZ8QcvMAAAAM/epic-fail-fall.gif'></center>

The situation quickly becomes frustrating for cat owners. Cats can no longer be left alone, as they are prone to forgetting where they put their toys and treats, and are even known to accidentally lock themselves in closets and bathrooms.

To address this problem, cat owners decide to deploy cameras equipped with machine vision to detect and track the activities of these forgetful felines. However, they want to make sure the algorithm can accurately identify cats and doesn't raise false alarms, especially while the owners are napping. To achieve this, they hire a machine learning expert(you, yes you) to interpret the algorithm.

## We need you! [TODO]

You are given a pre-trained ResNet model that is trained on Imagenet 1k dataset. Your task is to interpret "Why the ResNet model detects cats?"

For interpreting a classification task, there are multiple dimensions to choose from (Global vs Local, Model agnostic vs. specific, Inherent vs. post hoc). We will be using a Model agnostic post hoc method and deploy it at a local scale

Specifically, we will use LIME, SHAP, and integrated-gradient in this project. For each of these algorithms, you will be documenting the compute time and visualizing their explanations. At the end of the project, you'll be comparing the three evaluation approaches and assessing which you agree with most. So let's dive in!

## Setup
Before we start our mission, lets gets some gear set up. Firstly, lets install the missing packages and import the necessary libraries

### Installation of Libraries

In [ ]:
!pip install omnixai
!pip install dash
!pip install dash-bootstrap-components
## For local tunnel to a proxy server 
!npm install localtunnel

### Imports

First, we will import some usual suspects. We will use Pillow Image library to laod/create images. Finally, let us import our main weapon. Let us use [OmniXAI](https://opensource.salesforce.com/OmniXAI/latest/index.html) (Omni eXplainable AI), a Python library for explainable AI (XAI).

In [ ]:
## The usual suspects
import json
import numpy as np
import requests
import pickle

## To build our classifer
import torch
from torchvision import models, transforms

## Pillow Library Image function alias PilImage
from PIL import Image as PilImage

## Omnixai library to build our explainer
from omnixai.preprocessing.image import Resize
from omnixai.data.image import Image
from omnixai.explainers.vision import VisionExplainer
from omnixai.visualization.dashboard import Dashboard

## Image Data and Classifier

In [ ]:
## Let's start by loading the image that we want to explain
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
download = requests.get(url, stream=True).raw

## TODO: Read the image using Pillow and convert the image into RBG
### Hint: Use PilImage to read and convert

# image = Image(...)

In [ ]:
## TODO: Print the image shape and view the image

## Print the image shape
# print(...)

# Now, let's view it
image.to_pil()
# Shh! They are napping...

In [ ]:
## Before we build our classifier, lets make sure to setup the device.
## To run this notbeook via GPU: Edit->Notebook settings ->Hardware accelerator -> GPU
## If your GPU is working, device is "cuda"
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
## TODO: Lets build our classification model. We will use pre-trained ResNet34 model from PyTorch torchvision models.
## Make sure to load the model onto the device for gpu

# model = ...

In [ ]:
# Lets get a summary of our model using torchsummary
from torchsummary import summary
## TODO: Print the model summary
### Hint: Use image shape for input_size
# summary(...)

In [ ]:
## Did you notice the last layer had 1000 classes. Lets import all the classes. 
## We will later pass this to our explainer
classes_url = 'https://gist.githubusercontent.com/DaniFojo/dad37f5bf00ddeb56ed36daf561dbf69/raw/bd006b86300a5886ac7f897a44b0525b75a4b5a1/imagenet_labels.json'
imagenet_classes = json.loads(requests.get(classes_url).text)
idx2label =  {int(k):v for k,v in imagenet_classes.items()}

first_label = idx2label[next(iter(idx2label))]
print(f"The first class label from the ImageNet dataset is: '{first_label}'")

## Buiding our Explainer

To build our Explainer for our model, we will use [Vision Explainer](https://opensource.salesforce.com/OmniXAI/v1.2.3/omnixai.explainers.vision.html) by OmniXAI. The explainer needs some pre-processing and post-processing.

### Pre-processor

In [ ]:
## TODO: Build the pre-processor pipeline for the explainer

# The preprocessing function should convert the image to a Tensor 
# and then Normalise it

# 1. Compose the transformations
# transform = transforms.Compose([
    ## 1a. write code to convert the image to tensor
    #
    ## 1b. write code to normalize the image
    # 
# ])

In [ ]:
## TODO: Create the preprocess logic using the transformation built in previous cell
### Hint: Use torch.stack and load the images to the device

# def preprocess(images):
#   """
#   Args:
#     images: Sequence of images to preprocess using the composed 
#             transformations created above

#   Returns:  
#     preprocessed_images: Sequence of preprocessed images
#   """
#   preprocessed_images = ...
#   return preprocessed_images

### Post-processor

Next, we need to define our post-processing function:

In [ ]:
## TODO: Build the post-processor function for the explainer
# We will apply a softmax function to the logits obtained in the last layer
# in order to convert the prediction scores to probabilities

# def postprocess(logits):
#   """
#   Args:
#     logits: Logits from the last layer of the model
  
#   Returns:
#     postprocessed_outputs: Output from the Softmax layer applied to the logits
#   """

### Vision Explainer
Now, construct the explainer using the VisionExplainer class. You'll want to provide it a list of the three explainer types you'd like to try: LIME, SHAP, and integrated gradient. Be sure to check the documentation for the appropriate arguments! See the sample code for VisionExplainer [here](https://opensource.salesforce.com/OmniXAI/v1.2.3/tutorials/vision.html).

In [ ]:
#TODO: Build the VisionExplainer by filling in the blanks
# explainer = VisionExplainer(
#     explainers=[ ...],
#     mode="...",
#     model=...,
#     preprocess=...,
#     postprocess=...,

# )

Now, we can generate some explanations for each of the explainers using the explainer.explain() method:

In [ ]:
## Time to generate the explanations
local_explanations = explainer.explain(Image(
    data=np.concatenate([
        image.to_numpy()]),
    batched=True
))

In [ ]:
## Lets write the local_explantions to a pickle file. We will use this in our dashboard
with open('file.pkl', 'wb') as file:
    # A new file will be created
    pickle.dump(local_explanations, file)

## Dashboard
Now let's create a Dashboard to visualize our different explainers that we just built

In [ ]:
### Google Colab hosts the server on remote local. Therefore, localhost on your machine will not lead you to the dashboard

## Open `output.log` from files and use the link to get redirected. 
## <NOTE> : It might take a minute for the log file to show up. Hit refresh if need be.
!nohup npx localtunnel --port 8000 > output.log &

In [ ]:
##########################################################
###### Use the link from previous cell once running ######
##########################################################


## TODO: Fill in the Dashboard parameters

# dashboard = Dashboard(
#     instances=...,
#     local_explanations=...,
#     class_names= ...
# )


## Do not change the port number
## <NOTE> Once you open the link, it might take a minute or two for the website to load fully. Be patient :)
dashboard.show(port=8000)

## Outro

🎉Yay, you did it! Now that we've seen the explantions, you are ready to answer some questions about the various explanations!

1. What are your thoughts on Interpretable AI?
2. Compare the various explanations. Which method do you agree with most, why?
3. Do you think the ResNet model is good enough for cat owners?

## Bonus (Extension)
Document the computation time for each explainer: LIME, SHAP, and integrated-gradient.

In [ ]:
## Lets use hugging face cats vs dogs dataset
!pip install datasets

In [ ]:
## Now we will load 5 cat images from the dataset
from datasets import load_dataset

## Feel free to change this number. In order to not run out of RAM we use 5 images
NUM_IMAGES = 5
dataset = load_dataset("cats_vs_dogs")
cats_data = dataset['train'][0:NUM_IMAGES]['image']
cats_data

In [ ]:
## Notice that the image sizes are different. 
## TODO: Convert them to same size using transforms.Resize

#transform_resize = transforms.Compose([
#    transforms.Resize(...)
#])

In [ ]:
## Lets use the transformer and stack the images
# TODO: Use `transform_resize` and `np.stack`

# cats = ([... for cat in cats_data])

In [ ]:
## We will use this explainer function to create independant explainer 
def explainer(explainer):
  return VisionExplainer(
    explainers=[explainer],
    mode="classification",
    model=model,
    preprocess=preprocess,
    postprocess=postprocess,
  )

In [ ]:
### TODO: Initialize the explainer for 'Lime', 'SHAP', and 'integrated gradient'
# lime = explainer(...)
# shap = explainer(...)
# ig = explainer(...)

In [ ]:
## Let us time the results. We will use built-in magic commands in jupyter 
%time lime_results = lime.explain(cats)

In [ ]:
%time shap_results = shap.explain(cats)

In [ ]:
%time ig_results = ig.explain(cats)

In [ ]:
### Google Colab hosts the server on remote local. Therefore, localhost on your machine will not lead you to the dashboard

## Open `output.log` from files and use the link to get redirected. 
## <NOTE> : It might take a minute for the log file to show up. Hit refresh if need be.
!nohup npx localtunnel --port 8000 > output.log &

In [ ]:
## Combine all results
combine_results = lime_results
combine_results['shap'] = shap_results['shap']
combine_results['ig'] = ig_results['ig']

## Lets visualize the results on the Dashboard
dashboard = Dashboard(
    instances=Image(cats,batched =True),
    local_explanations=combine_results,
    class_names=idx2label
)
## Do not change the port
## <NOTE> Once you open the link, it might take a minute or two for the website to load fully. Be patient :)
dashboard.show(port=8000)

## Final Thoughts🎉

Congratulations on finishing the bonus sections. It is an impressive feat!

---
Please share your observations about the computation time for each of the explainers and recommend a method based on this and any other relevant factors, such as effectiveness or accuracy? If your recommendation differs from a previous suggestion, please explain the reason for this change.